In [1]:
from ngsolve import *
from ngsolve.meshes import Make1DMesh
from ngsolve.webgui import Draw
import numpy as np

importing NGSolve-6.2.2204


In [6]:
axial_load = 4.1*pi**2
lamda_1 = 2*pi
c = 2*np.sqrt((axial_load)/(lamda_1**2)-1)


In [ ]:
mesh = Make1DMesh(10, mapping=None, periodic=False)
Draw(mesh)
# help(Make1DMesh)

In [ ]:
V = H1(mesh, order=2, dirichlet="left|right")
Q = NumberSpace(mesh)
fesm = V*Q
print ("ndof =", fesm.ndof)

In [ ]:
psi, mu = fesm.TrialFunction()
v , lamda = fesm.TestFunction()

In [ ]:
am = BilinearForm(fesm)
am += (grad(psi)*grad(v)+1/2*grad(psi)*mu*grad(v) + mu*lamda - grad(psi)**2*lamda)*dx

gfu = GridFunction(fesm)

In [ ]:
def SimpleNewtonSolve(gfu,am,tol=1e-13,maxits=25):
    res = gfu.vec.CreateVector()
    du = gfu.vec.CreateVector()
    fes = gfu.space
    for it in range(maxits):
        print ("Iteration {:3}  ".format(it),end="")
        am.Apply(gfu.vec, res)
        am.AssembleLinearization(gfu.vec)
        du.data = am.mat.Inverse(fes.FreeDofs()) * res
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break

In [ ]:
gfu.vec[:] = 1
SimpleNewtonSolve(gfu,am)